In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
civic_df = pd.read_csv("rawData/CIViC/nightly-ClinicalEvidenceSummaries.tsv", sep='\t')
# print(civic_df)
civic_df.head()

,molecular_profile,molecular_profile_id,disease,doid,phenotypes,therapies,therapy_interaction_type,evidence_type,evidence_direction,evidence_level,...,citation,nct_ids,rating,evidence_status,evidence_id,variant_origin,last_review_date,evidence_civic_url,molecular_profile_civic_url,is_flagged
0,JAK2 V617F,64,Lymphoid Leukemia,1037.0,NaN,NaN,NaN,Diagnostic,Supports,B,...,"Levine et al., 2005, Blood",NaN,4.0,accepted,1,Somatic,2023-01-09 21:46:26 UTC,https://civicdb.org/links/evidence_items/1,https://civicdb.org/links/molecular_profiles/64,False
1,PDGFRA D842V,99,Gastrointestinal Stromal Tumor,9253.0,NaN,NaN,NaN,Diagnostic,Supports,B,...,"Lasota et al., 2004, Lab. Invest.",NaN,3.0,accepted,2,Somatic,2023-01-09 21:46:27 UTC,https://civicdb.org/links/evidence_items/2,https://civicdb.org/links/molecular_profiles/99,False
2,DNMT3A R882,32,Acute Myeloid Leukemia,9119.0,NaN,NaN,NaN,Diagnostic,Supports,B,...,"LaRochelle et al., 2011, Oncotarget",NaN,2.0,accepted,3,Somatic,2023-01-09 21:46:25 UTC,https://civicdb.org/links/evidence_items/3,https://civicdb.org/links/molecular_profiles/32,False
3,DNMT3A R882,32,Acute Myeloid Leukemia,9119.0,NaN,NaN,NaN,Diagnostic,Supports,B,...,"Ribeiro et al., 2012, Blood",NaN,3.0,accepted,4,Somatic,2023-01-09 21:46:25 UTC,https://civicdb.org/links/evidence_items/4,https://civicdb.org/links/molecular_profiles/32,False
4,JAK2 V617F,64,Chronic Myeloid Leukemia,8552.0,NaN,NaN,NaN,Diagnostic,Supports,B,...,"Levine et al., 2005, Blood",NaN,4.0,accepted,5,Somatic,2023-01-09 21:46:26 UTC,https://civicdb.org/links/evidence_items/5,https://civicdb.org/links/molecular_profiles/64,False


In [3]:
# civic_df['evidence_direction'].value_counts()

predictive_df = civic_df.dropna(subset=['therapies'])
print(predictive_df.shape)
support_df = predictive_df[predictive_df['evidence_direction'] == 'Supports']
print(support_df.shape)

(2562, 25)
(2197, 25)


In [4]:
####
variant_df = pd.read_csv("rawData/CIViC/nightly-VariantSummaries.tsv", sep='\t')

In [5]:
missense_df = variant_df.dropna(subset=['start','stop'])
columns = ['gene', 'variant', 'drugs', 'clinical_significance', 'chromosome', 'start', 'stop', 'reference_bases', 'variant_bases', 'representative_transcript', 'drug_interaction_type']
missense_df = pd.DataFrame(data=missense_df, columns=columns)
missense_df.drop_duplicates(inplace=True)

# check missense
missense_df['variant'] = missense_df['variant'].str.split(' ')
missense_df['variant'] = missense_df['variant'].apply(lambda x: x[-1])
missense_df['variant'] = missense_df['variant'].str.split('::')
missense_df['variant'] = missense_df['variant'].apply(lambda x: x[-1])

pd.options.display.max_rows = 20
# update: regular expression filtering

missense_df = missense_df[missense_df['variant'].str.contains('[A-Z][0-9]+[A-Z]')]
missense_df = missense_df[~missense_df['variant'].str.contains('_')]
missense_df = missense_df[~missense_df['variant'].str.contains('DEL')]
missense_df = missense_df[~missense_df['variant'].str.contains('INS')]
missense_df = missense_df[~missense_df['variant'].str.contains('INST')]
missense_df = missense_df[~missense_df['variant'].str.contains('-')]
missense_df = missense_df[~missense_df['variant'].str.contains('/')].reset_index(drop=True)

missense_df

,gene,variant,drugs,clinical_significance,chromosome,start,stop,reference_bases,variant_bases,representative_transcript,drug_interaction_type
0,ABL1,T315I,NaN,NaN,9,133748283.0,133748283.0,C,T,ENST00000318560.5,NaN
1,ABL1,E255K,NaN,NaN,9,133738363.0,133738363.0,G,A,ENST00000318560.5,NaN
2,AKT1,E17K,NaN,NaN,14,105246551.0,105246551.0,C,T,ENST00000407796.2,NaN
3,ALK,F1174L,NaN,NaN,2,29443695.0,29443695.0,G,T,ENST00000389048.3,NaN
4,ALK,R1275Q,NaN,NaN,2,29432664.0,29432664.0,C,T,ENST00000389048.3,NaN
...,...,...,...,...,...,...,...,...,...,...,...
455,ALK,L1196M,NaN,NaN,2,29443631.0,29443631.0,G,T,ENST00000389048.3,NaN
456,ALK,C1156Y,NaN,NaN,2,29445258.0,29445258.0,C,T,ENST00000389048.3,NaN
457,EZH2,A692V,NaN,NaN,7,148506437.0,148506437.0,G,A,ENST00000320356.2,NaN
458,EZH2,A682G,NaN,NaN,7,148506467.0,148506467.0,G,C,ENST00000320356.2,NaN


In [6]:
missense_df['drugs'].value_counts()


Series([], Name: drugs, dtype: int64)